<a href="https://colab.research.google.com/github/amimoon/AI_DeepLearning_Personal_Project/blob/main/TermProject_flask_app_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask Website Example

You can develop a simple web server using Colab and python Flask framework. Vanilla Colab cannot be used for public deployement, but we can build a temporary server using Flask-Ngrok library.

In [ ]:
!pip install flask-ngrok

In [ ]:
'''
import shutil

shutil.rmtree('/content/DeepLearning_TermProject/', ignore_errors=False)
'''

"\nimport shutil\n\nshutil.rmtree('/content/DeepLearning_TermProject/', ignore_errors=False)\n"

In [ ]:
%cd /content

/content


# Github의 레파지토리 Clone

flask에서 사용하는 폴더를  Github에 배포하고 이를 복제하여 활용

In [ ]:
import os
if not os.path.exists('DeepLearning_TermProject'):
  !git clone https://github.com/MSD-AI/DeepLearning_TermProject.git

Cloning into 'DeepLearning_TermProject'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 256 (delta 104), reused 156 (delta 46), pack-reused 0
Receiving objects: 100% (256/256), 46.61 MiB | 35.48 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [ ]:
%cd /content/DeepLearning_TermProject

/content/DeepLearning_TermProject


In [ ]:
import new_style_transfer
from flask import Flask, redirect, url_for, render_template, session, request
#from flask import escape, request, Response, g, make_response
from datetime import timedelta

from flask_ngrok import run_with_ngrok
#import os, sys
from flask.templating import render_template

* Flask를 실행하기 위한 정보 셋팅
* 

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
app.secret_key = "thisisasecret"
app.permanent_session_lifetime = timedelta(minutes=5)

* 초기화면에 사용한 index.html 파일 구성
* 이미지변환 버튼 클릭 시 new_style_transfer.py가 호출되어 이미지 변환작업 실행
* 부트 스트랩을 이용하여 화면 Style 구성

In [ ]:
text = '''
<!DOCTYPE html>
<html lang="ko">
    <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>Term Project</title>
        <!-- Favicon-->
        <link rel="icon" type="image/x-icon" href="../static/favicon.ico" />
        <!-- Core theme CSS (includes Bootstrap)-->
        <link rel=stylesheet href="../static/styles.css" />
        <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js" integrity="sha384-J6qa4849blE2+poT4WnyKhv5vZF5SrPo0iEjwBvKU7imGFAV0wwj1yYfoRSJoZ+n" crossorigin="anonymous"></script>
        <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js" integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo" crossorigin="anonymous"></script>
        <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js" integrity="sha384-wfSDF2E50Y2D1uUdj0O3uMBJnjuUD4Ih7YwaYd1iqfktj0Uod8GCExl3Og8ifwB6" crossorigin="anonymous"></script>
        <script>
            $(function(){
            $(window).scroll(function(){
                var navbar = $(this).scrollTop();
                console.log(navbar);
                // var $header = $('header');
                var $navbar_tag = $('.navbar')
                if(navbar > 10){
                    $navbar_tag.addClass('activated');
                }
                else{
                    $navbar_tag.removeClass('activated');
                    }
                })
            })
        </script>
        <script type="text/javascript">
            function previewImage(targetObj, view_area) {
            var preview = document.getElementById(view_area); //div id
            var ua = window.navigator.userAgent;

            var files = targetObj.files;
            for ( var i = 0; i < files.length; i++) {
                var file = files[i];
                var imageType = /image.*/; //이미지 파일일경우만.. 뿌려준다.
                if (!file.type.match(imageType))
                    continue;
                var prevImg = document.getElementById("prev_" + view_area); //이전에 미리보기가 있다면 삭제
                if (prevImg) {
                    preview.removeChild(prevImg);
                }
                var img = document.createElement("img"); 
                img.id = "prev_" + view_area;
                img.classList.add("obj");
                img.file = file;
                img.style.width = '700px'; 
                img.style.height = '400px';
                preview.appendChild(img);
                if (window.FileReader) { // FireFox, Chrome, Opera 확인.
                    var reader = new FileReader();
                    reader.onloadend = (function(aImg) {
                        return function(e) {
                            aImg.src = e.target.result;
                        };
                    })(img);
                    reader.readAsDataURL(file);
                } else { // safari is not supported FileReader
                    //alert('not supported FileReader');
                    if (!document.getElementById("sfr_preview_error_"
                            + view_area)) {
                        var info = document.createElement("p");
                        info.id = "sfr_preview_error_" + view_area;
                        info.innerHTML = "not supported FileReader";
                        preview.insertBefore(info, null);
                    }
                }
            }
        }
        </script>
    </head>
    <body>
        <!-- Responsive navbar-->
        <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
            <div class="container">
                <a class="navbar-brand" href="/">MSD.ai</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation"><span class="navbar-toggler-icon"></span></button>
                <div class="collapse navbar-collapse" id="navbarSupportedContent">
                    <ul class="navbar-nav ms-auto mb-2 mb-lg-0">
                        <li class="nav-item"><a class="nav-link active" aria-current="page" href="/">Home</a></li>
                        <li class="nav-item"><a class="nav-link" href="/company">Company</a></li>
                        <li class="nav-item"><a class="nav-link" href="/product"">Product</a></li>
                    </ul>
                </div>
            </div>
        </nav>
        <!-- Header - set the background image for the header in the line below-->
        <header class="py-5 bg-image-full" style="background-image: url('https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/Vincent_van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/776px-Vincent_van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')">
            <div class="text-center my-5">
                <h1 class="text-white fs-3 fw-bolder">Nueral Style Transfer Project</h1>
                <p class="text-white mb-0">경영자를 위한 딥러닝 </p>
            </div>
        </header>

		<!-- py model로 보낼 정보 -->
        <form class=form-select align="center" action="/nst_post" method="POST" enctype="multipart/form-data">
            <!--Reference Image 선택하기-->
            <h2 align="center">Reference Images</h2>
            <p class='mb-1'> 스타일을 참조할 이미지를 선택해주세요.</p>
            <table align="center">
                <tr>
                    <td><img class="refer_img" id="refer_img1" src="../static/images/nst_get/rain_princess.JPG"></td>
                    <td><img class="refer_img" id="refer_img2" src="../static/images/nst_get/the_stary_night.JPG"></td>
                    <td><img class="refer_img" id="refer_img3" src="../static/images/nst_get/scream.JPG"></td>
                    <td><img class="refer_img" id="refer_img3" src="../static/images/nst_get/zentangle_art.JPG"></td>
                </tr>
                <tr>
                    <td><input type="radio" name="refer_img" value="rain_princess.JPG"></td>
                    <td><input type="radio" name="refer_img" value="the_stary_night.JPG"></td>
                    <td><input type="radio" name="refer_img" value="scream.JPG"></td>
                    <td><input type="radio" name="refer_img" value="zentangle_art.JPG"></td>
                </tr>
            </table>
            
            <!--사용자 이미지 업로드-->
            <h2 align="center">Target Image</h2>
            <p class='mb-0'>스타일을 바꿀 이미지를 업로드해주세요.</p>
                <div align="center" id='view_area'></div>
            <br>
            <div>
                <input type="file" name="user_img" id="user_img" value="userIMgage" onchange="previewImage(this,'view_area')"/>
                <input type="submit" value="이미지변환"/>
            </div>
            <!--
                <input type="submit" value="이미지 변환하기"/><br><br>
                <div><button type="button" class="btn btn-primary">이미지변환</button></div>
            -->
        </form>
        <!-- Footer-->
        <footer class="py-4 bg-dark">
            <div class="container"><p class="m-0 text-center text-white">Copyright &copy; MSD.AI 2021</p></div>
        </footer>  
        <!-- Bootstrap core JS-->
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.1/dist/js/bootstrap.bundle.min.js"></script>
    </body>
</html>
'''
file = open("templates/index.html","w")
file.write(text)
file.close()

* 초기화면에 사용한 nst_post.html 파일 구성
* 초기화면에서 이미지 변환 버튼 클릭 시 이동되며 스타일 이미지, 사용자 선택 이미지, 변환된 이미지를 출력
* 부트 스트랩을 이용하여 화면 Style 구성

In [ ]:
  text = '''
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>Term Project</title>
        <!-- Favicon-->
        <link rel="icon" type="image/x-icon" href="./static/favicon.ico" />
        <!-- Core theme CSS (includes Bootstrap)-->
        <link rel=stylesheet href="./static/styles.css" />
    </head>
    <body>
        <!-- Responsive navbar-->
        <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
            <div class="container">
                <a class="navbar-brand" href="/">MSD.AI</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation"><span class="navbar-toggler-icon"></span></button>
                <div class="collapse navbar-collapse" id="navbarSupportedContent">
                    <ul class="navbar-nav ms-auto mb-2 mb-lg-0">
                        <li class="nav-item"><a class="nav-link active" aria-current="page" href="/">Home</a></li>
                        <li class="nav-item"><a class="nav-link" href="/company">Company</a></li>
                        <li class="nav-item"><a class="nav-link" href="/product"">Product</a></li>
                    </ul>
                </div>
            </div>
        </nav>
        <!-- Header - set the background image for the header in the line below-->
        <header class="py-5 bg-image-full" style="background-image: url('https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/Vincent_van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/776px-Vincent_van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')">
            <div class="text-center my-5">
                <h1 class="text-white fs-3 fw-bolder">Nueral Style Transfer Project</h1>
                <p class="text-white-50 mb-0">경영자를위한딥러닝 </p>
            </div>
        </header>

	
        <!-- 선택한 Refer/Target 이미지 및 Transfer 이미지 -->
        <div class="side-page-main-div">
            <table align="center">
                <tr>
                    <td><h2 align="center">Reference Image</h2></td>
                    <td><h2 align="center">Target Image</h2></td>
                </tr>
                <tr align="center">
                    <td><img class='nst_img' src="{{refer_img}}" alt='reference image'></td>
                    <td><img class='nst_img' src="{{user_img}}" alt='target image'></td>
            </table>
            <table align="center">
                </tr>
                    <td><h2 align="center">Transfer Image</h2></td>				     
                <tr>
                    <td align="center"><img class="nst_result_img" src="{{transfer_img}}"alt='transfer image'></td>
                </tr>
            </table>
        </div>
        <br>
        <!-- Footer-->
        <footer class="py-4 bg-dark">
            <div class="container"><p class="m-0 text-center text-white">Copyright &copy; MSD.AI 2021</p></div>
        </footer>  
        <!-- Bootstrap core JS-->
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.1/dist/js/bootstrap.bundle.min.js"></script>
    </body>
</html>
'''
file = open("templates/nst_post.html","w")
file.write(text)
file.close()

A function for home "/" , "/company", "/product" route.

In [ ]:
# Main page
@app.route('/')
def index():
    return render_template('index.html')
    
@app.route('/company')
def company():
    return render_template('company.html')

@app.route('/product')
def product():
    return render_template('product.html')

A function for "/nst_post" route.

In [ ]:
@app.route('/nst_post', methods=['GET','POST'])
def nst_post():
  if request.method == 'POST':
    #root_path()

    # Reference Image
    refer_img = request.form['refer_img']
    refer_img_path = './static/images/nst_get/'+str(refer_img)

    # User Image (target image)
    user_img = request.files['user_img']
    user_img.save('./static/images/nst_get/user/'+str(user_img.filename))
    user_img_path = './static/images/nst_get/user/'+str(user_img.filename)

    # Neural Style Transfer 
    #transfer_img = neural_style_transfer.main(refer_img_path, user_img_path)
    transfer_img = new_style_transfer.main(refer_img_path, user_img_path)
    transfer_img_path = './static/images/nst_result/'+str(transfer_img.split('/')[-1])

  return render_template('nst_post.html', 
          refer_img=refer_img_path, user_img=user_img_path, transfer_img=transfer_img_path)

Run python app.  The "ngrok" library provides public address that anyone can visit.

In [ ]:
if __name__ == "__main__":
	app.run()
  #app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ac4cf24b84e7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Jun/2021 13:04:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:43] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:44] "GET /static/images/nst_get/the_stary_night.JPG HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:44] "GET /static/images/nst_get/rain_princess.JPG HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:44] "GET /static/images/nst_get/scream.JPG HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:44] "GET /static/images/nst_get/zentangle_art.JPG HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:45] "GET /static/favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:47] "GET /company HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:47] "GET /static/images/company1.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:04:47] "GET /static/images/company2.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:05:25] "GET /company HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 13:05:29] "GET /product HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun